In [2]:
from tensorflow.keras.datasets import mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [5]:
import torch as t

class RNN(t.nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.lstm = t.nn.LSTM(28, 128, 2, batch_first=True)
        self.fc = t.nn.Linear(128, 10)
    
    def forward(self, inputs):
        x, _ = self.lstm(inputs)
        x = x[:, -1, :] # 取最后一层的输出
        out = self.fc(x)
        return out
net = RNN()

net

RNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [6]:
device = 'cude:0' if t.cuda.is_available() else 'cpu'

In [7]:
loss_func = t.nn.CrossEntropyLoss()
optimizer = t.optim.Adam(net.parameters(), lr=1e-3)

In [8]:
from torch.utils.data import Dataset

In [9]:
class MnistDataLoader(Dataset):
    def __init__(self):
        self.x_train = x_train
        self.y_trian = y_train
        
    def __len__(self):
        return len(self.y_train)
    
    def __getitem__(self, idx):
        return self.x_train[idx], self.y_train[idx]
    

In [10]:
from torch.utils.data import DataLoader, TensorDataset  

In [11]:
mnistDatasets = TensorDataset(t.from_numpy(x_train).to(device), t.from_numpy(y_train).to(device))

In [12]:
mnisDataLoader = DataLoader(mnistDatasets, batch_size=32, shuffle=True)

In [ ]:
running_loss = 0

for idx, data in enumerate(mnisDataLoader, 1):
    x_, y_ = data # x_ dtype=torch.uint8
    y_pred = net(x_.float())
    loss = loss_func(y_pred, y_.long())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
    
    if idx % 100 == 0:
        print("idx=%d loss=%.3f" % (idx, running_loss / 100))
        running_loss = 0

idx=100 loss=1.313
idx=200 loss=0.551


In [ ]:
mnistTestData = TensorDataset(t.from_numpy(x_test), t.from_numpy(y_test))

mnistTestDataLoader = DataLoader(mnistTestData, batch_size=32)

In [ ]:
net.eval()

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
acc_num = 0

for data in tqdb(mnistTestDataLoader):
    x_test_, y_test_ = data
    y_test_pred = t.argmax(net(x_test_.float()), dim=1)
    
    acc_num  = acc_num + (y_test_pred == y_test_).sum()

acc_num.item() / 10000

In [ ]:
t.argmax(y_pred, dim=1)

In [ ]:
(y_ == t.argmax(y_pred, dim=1)).sum()

In [ ]:
(y_test_pred == y_test_).sum() 

In [ ]:
y_test_